First updated @ Nov 2, 2021

Last updated @ Nov 2, 2021

In [38]:
import pandas as pd
import ast
from collections import defaultdict
import numpy as np

## Load 3 datasets

- The Bechdel data (only has imdb id)
- The credits data (only has tmdb id)
- The links data, which has both imdb id and tmdb id, so that we can join the three datasets together to get the movies we want.

In [9]:
# load bechdel data
bechdel_df = pd.read_csv('./Bechdel_detailed.csv', index_col=0)
bechdel_df

,year,rating,dubious,imdbid,id,submitterid,date,visible
title,,,,,,,,
Passage de Venus,1874.0,0.0,0.0,3155794.0,9602.0,18880.0,2021-04-02 20:58:09,1.0
La Rosace Magique,1877.0,0.0,0.0,14495706.0,9804.0,19145.0,2021-05-11 00:11:22,1.0
Sallie Gardner at a Gallop,1878.0,0.0,0.0,2221420.0,9603.0,18882.0,2021-04-03 02:25:27,1.0
Le singe musicien,1878.0,0.0,0.0,12592084.0,9806.0,19151.0,2021-05-11 23:38:54,1.0
Athlete Swinging a Pick,1881.0,0.0,0.0,7816420.0,9816.0,19162.0,2021-05-13 01:32:14,1.0
...,...,...,...,...,...,...,...,...
"Guilty, The",2021.0,1.0,0.0,9421570.0,10079.0,19599.0,2021-10-10 20:16:13,1.0
Till Death,2021.0,0.0,0.0,11804152.0,10083.0,19613.0,2021-10-18 19:58:48,1.0
"Last Duel, The",2021.0,3.0,1.0,4244994.0,10085.0,19619.0,2021-10-20 02:30:29,1.0


In [3]:
# Load movie credits data
credits_df = pd.read_csv('../TheMoviesData/credits.csv')
credits_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [11]:
# load imdb id and tmdb id data
links_df = pd.read_csv('../TheMoviesData/links.csv', index_col=0)
links_df

,imdbId,tmdbId
movieId,,
1,114709,862.0
2,113497,8844.0
3,113228,15602.0
4,114885,31357.0
5,113041,11862.0
...,...,...
176269,6209470,439050.0
176271,2028550,111109.0
176273,303758,67758.0


## Filter credits_df with bechdel_df

In [44]:
for index, imdbid in bechdel_df['imdbid'].iteritems():
    matching_tmdbid = links_df[links_df['imdbId'] == imdbid]['tmdbId']
    # if there is indeed a matching id
    if len(matching_tmdbid) == 1:
        matching_tmdbid = matching_tmdbid.item()
    # if there is not
    else:
        matching_tmdbid = np.nan
    bechdel_df.at[index, 'tmdbid'] = matching_tmdbid

In [48]:
bechdel_df.tail(10)

,year,rating,dubious,imdbid,id,submitterid,date,visible,tmdbid
title,,,,,,,,,
Dune,2021.0,2.0,0.0,1160419.0,10052.0,19545.0,2021-09-16 08:04:46,1.0,NaN
#Nofilter,2021.0,0.0,0.0,14843560.0,10065.0,19562.0,2021-09-20 02:39:47,1.0,NaN
CODA,2021.0,3.0,0.0,10366460.0,10076.0,19587.0,2021-10-04 21:49:55,1.0,NaN
No Time To Die,2021.0,2.0,1.0,2382320.0,10075.0,19585.0,2021-10-04 08:29:46,1.0,NaN
"Many Saints of Newark, The",2021.0,2.0,0.0,8110232.0,10074.0,19583.0,2021-10-03 22:25:04,1.0,NaN
"Guilty, The",2021.0,1.0,0.0,9421570.0,10079.0,19599.0,2021-10-10 20:16:13,1.0,NaN
Till Death,2021.0,0.0,0.0,11804152.0,10083.0,19613.0,2021-10-18 19:58:48,1.0,NaN
"Last Duel, The",2021.0,3.0,1.0,4244994.0,10085.0,19619.0,2021-10-20 02:30:29,1.0,NaN
Venom Let there be Carnage,2021.0,2.0,0.0,7097896.0,10092.0,19627.0,2021-10-21 10:11:03,1.0,NaN


In [47]:
bechdel_df['tmdbid'].count()

7578